In [2]:
from decouple import Config, RepositoryEnv
from anthropic import Anthropic

DOTENV_FILE = '.env'
cfg = Config(RepositoryEnv(DOTENV_FILE))
anthropic_key = cfg('ANTHROPIC_KEY')
anthropic_client = Anthropic(
    api_key= anthropic_key,
)

claude_haiku = 'claude-3-haiku-20240307'
claude_opus = 'claude-3-opus-20240229'
claude_sonnet = 'claude-3-sonnet-20240229'

def claude_stream(*, model, max_tokens, temperature, system, messages):
    # rendered_messages = []
    # newline2 = '\n\n'
    # roleprefix = '####################################\n'
    # for msg in messages:
    #     text_blocks = []
    #     for ct in msg['content']:
    #         if ct['type'] != 'text':
    #             print('! Warning: ignoring non-text content')
    #             continue
    #         text_blocks.append(ct['text'])
    #     message_content = ''.join(text_blocks)
    #     rendered_messages.append(f'{roleprefix}{msg["role"].upper()}: {message_content}')
    # rendered = f'{roleprefix}SYSTEM: {system}{newline2}{newline2.join(rendered_messages)}'
    # print(rendered)
    totalResult = ''
    with anthropic_client.messages.stream(
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        system=system,
        messages=messages
    ) as stream:
        print('Assistant: ', end="", flush=True)
        for text in stream.text_stream:
            totalResult += text
            print(text, end="", flush=True)
        print('\n')
    return totalResult


In [3]:
system_prompt = ''

# botreply = claude_stream(
#     model=claude_opus,
#     max_tokens=4096,
#     temperature=0,
#     system=system_prompt,
#     messages=[
#         {
#             'role': 'user',
#             'content': [
#                 {
#                     'type': 'text',
#                     'text': 'Hello world',
#                 }
#             ]
#         }
#     ]
# )

In [4]:
from typing import Literal, List, Dict
from dataclasses import dataclass

@dataclass
class Message:
    role: Literal['user', 'assistant']
    text: str

@dataclass
class Transcript:
    messages: List[Message]

    def to_messages(self):
        return [{
            'role': m.role,
            'content': [
                {
                    'type': 'text',
                    'text': m.text
                }
            ]
        } for m in self.messages]

def run_anthropic_chat_loop():
    transcript = Transcript(messages=[])
    while True:
        while True:
            user_msg = input()
            if user_msg == 'exit':
                return
            elif len(user_msg) > 0:
                break
        print(f'User: {user_msg}\n')
        transcript.messages.append(Message(role='user', text=user_msg))
        botreply = claude_stream(
            model=claude_opus,
            max_tokens=4096,
            temperature=0,
            system=system_prompt,
            messages=transcript.to_messages(),
        )
        transcript.messages.append(Message(role='assistant', text=botreply))


In [5]:
run_anthropic_chat_loop()

User: say hi

Assistant: Hi there! How can I assist you today?



In [6]:
# Basic OpenAI usage

from openai import OpenAI

client = OpenAI(api_key=cfg('OPENAI_KEY'))

stream = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Say this is a test 10 times"}],
    stream=True,
)
for chunk in stream:
    # print(chunk.choices)
    if len(chunk.choices) == 0:
        continue
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)



This is a test. This is a test. This is a test. This is a test. This is a test. This is a test. This is a test. This is a test. This is a test. This is a test.

In [22]:
import asyncio
import time

async def mirror(s):
    time.sleep(2)
    # await asyncio.sleep(2)
    return s


# res = asyncio.gather(*[mirror(i) for i in range(5)])
# print(await res)

print(await asyncio.gather(mirror(1), mirror(2)))

[1, 2]


In [23]:
from anthropic import AsyncAnthropic

client = AsyncAnthropic(api_key=anthropic_key)

stream = await client.messages.create(
    model=claude_opus,
    max_tokens=4096,
    temperature=0,
    system='',
    messages=[
        {
            "role": "user",
            "content": "Hello, Claude",
        }
    ],
    stream=True,
)
async for event in stream:
    print(event)




message_start
content_block_start
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_delta
content_block_stop
message_delta
message_stop
